Tool Auto Export Data from Purchase Info Record Template to XML & upload to LTMC

In [48]:
# Getting and Knowing Data
import pandas as pd
import numpy as np
from datetime import datetime
import time

PURCHASE INFO RECORD DATA

In [58]:
# Step 1: Import Dataset
file_excel = r"C:\Users\Admin\Desktop\List AF.xlsx"
df_data = pd.read_excel(file_excel, usecols="A:L", skiprows=[1], keep_default_na=False)
df_data.head()

,MATNR,LIFNR,EKORG,WERKS,MEINS,KBETR_EXT,KPEIN,KMEIN,UMREN,UMREZ,VALID_FROM,VALID_TO
0,22H14C00599530001,2120,2300,2300,EA,11000,1,KG,3556.0,1000,01.11.2022,31.12.9999
1,22H14C00599530002,2120,2300,2300,EA,11000,1,KG,2460.0,1000,01.11.2022,31.12.9999
2,22H14C00599530003,2120,2300,2300,EA,11000,1,KG,5563.0,1000,01.11.2022,31.12.9999
3,22H14C00599530004,2120,2300,2300,EA,11000,1,KG,4450.0,1000,01.11.2022,31.12.9999
4,22H14C00599530005,2120,2300,2300,EA,11000,1,KG,4488.0,1000,01.11.2022,31.12.9999


LTMC_PIR_XMLSHEET.


In [59]:
path_pir = r"C:\Users\Admin\Desktop\LTMC_PurchaseInfoRecord.xlsx"
df_GD = pd.read_excel(path_pir, sheet_name='General Data',
                      header=4, skiprows=[5, 6, 7], keep_default_na=False, na_filter="")
df_POD = pd.read_excel(path_pir, sheet_name='Purchasing Organization Data',
                       header=4, skiprows=[5, 6, 7], keep_default_na=False)
df_cdt = pd.read_excel(path_pir, sheet_name='Conditions',
                       header=4, skiprows=[5, 6, 7], keep_default_na=False)


DEFAULD DATA

In [60]:
# For Subcontract Purchase info record.
# Sheet General Data
df_GD.MAHN1 = df_GD.MAHN2 = df_GD.MAHN3 = 0
# Sheet Purchasing Organization Data
df_POD.ESOKZ = df_cdt.ESOKZ = 3
df_POD.EKGRP = 105
df_POD.WAERS = df_cdt.KONWA = 'VND'
df_POD.MINBM = df_POD.MHDRZ = df_POD.BSTMA = 0
df_POD.NORBM = 1
df_POD.APLFZ = 1
df_POD.UEBTO = 0
df_POD.UNTTO = 0
df_POD.BWTAR = "EX"
df_POD.WEBRE = "X"
df_POD.MWSKZ = "VA"
df_POD.BSTAE = "0004"
# Sheet Conditions
df_cdt.COND_TYPE = 'PB00'
df_cdt.KOPOS = 1


For Loops & Fill Data


In [61]:
# Create sequence number
seq = str(datetime.now().strftime('%Y%m%d-%H%M%S')) + "-"
for index, row in df_data.iterrows():
    # Legacy Number
    df_GD.at[index, 'INFNR'] = df_POD.at[index,
                                         'INFNR'] = df_cdt.at[index, 'INFNR'] = str(seq + str(index))
for i in df_data.columns:
    if i in list(df_GD.columns):
        df_GD[i] = df_data[i]
    if i in list(df_POD.columns):
        df_POD[i] = df_data[i]
    if i in list(df_cdt.columns):
        df_cdt[i] = df_data[i]
        
df_cdt.KUMNE = df_data.UMREZ
df_cdt.KUMZA = df_data.UMREN

df_cdt = df_cdt.fillna("")
df_GD = df_GD.fillna("")
df_POD = df_POD.fillna("")

C:\Users\Admin\AppData\Local\Temp\ipykernel_20436\1170108722.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_cdt.KUMNE = df_data.UMREZ
C:\Users\Admin\AppData\Local\Temp\ipykernel_20436\1170108722.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_cdt.KUMZA = df_data.UMREN


GET HEADER SHEET

In [62]:
print(df_GD.shape , df_POD.shape, df_cdt.shape)

(11800, 27) (11800, 37) (11800, 19)


EXPORT TO EXCEL FILE

Put multiple dataframes across separate tabs/sheets

In [63]:
# function
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=8 , startcol=0, index= False)   
    writer.close()

# list of dataframes and sheet names
dfs = [df_GD, df_POD, df_cdt]
sheets = ['General Data','Purchasing Organization Data','Conditions']    

# run function
dfs_tabs(dfs, sheets, path_pir)